In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("blastchar/telco-customer-churn")

print("Path to dataset files:", path)

c:\Users\Nichole\Desktop\data-driven-process\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Nichole\.cache\kagglehub\datasets\blastchar\telco-customer-churn\versions\1


In [3]:
import pandas as pd
import os

file_path = os.path.join(path, "WA_Fn-UseC_-Telco-Customer-Churn.csv")
df = pd.read_csv(file_path)
print(df.head())

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

In [25]:
# Quantidade de clientes que cancelaram o serviço e que não cancelaram
churn_counts = df['Churn'].value_counts()
print("Cancelados:\n", churn_counts)

# porcentagem de clientes que cancelaram o serviço
churn_percentage = df['Churn'].value_counts(normalize=True) * 100
print("Porcentagem de cancelados:\n", churn_percentage.round(2))

Cancelados:
 Churn
No     5174
Yes    1869
Name: count, dtype: int64
Porcentagem de cancelados:
 Churn
No     73.46
Yes    26.54
Name: proportion, dtype: float64


In [30]:
# O objetivo é analisar os possíveis motivos que levam um cliente a cancelar o serviço identificando padrões de comportamento entre os clientes que permanecem e os que cancelam.

# Primeiro, vou criar duas variáveis separadas: uma para os clientes que cancelaram o serviço e outra para os que não cancelaram.
churned_customers = df[df['Churn'] == 'Yes']

# Somar a quantidade de clientes que cancelaram o serviço
sum_churned = churned_customers.shape[0]
print("Total de clientes que cancelaram o serviço:", sum_churned)

retained_customers = df[df['Churn'] == 'No']
# Somar a quantidade de clientes que não cancelaram o serviço
sum_retained = retained_customers.shape[0]
print("Total de clientes que não cancelaram o serviço:", sum_retained)

print("Churned Customers Shape:\n", churned_customers.head(2))

print("Retained Customers Shape:\n", retained_customers.head(2))

Total de clientes que cancelaram o serviço: 1869
Total de clientes que não cancelaram o serviço: 5174
Churned Customers Shape:
    customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
2  3668-QPYBK    Male              0      No         No       2          Yes   
4  9237-HQITU  Female              0      No         No       2          Yes   

  MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
2            No             DSL            Yes  ...               No   
4            No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract PaperlessBilling  \
2          No          No              No  Month-to-month              Yes   
4          No          No              No  Month-to-month              Yes   

      PaymentMethod MonthlyCharges  TotalCharges Churn  
2      Mailed check          53.85        108.15   Yes  
4  Electronic check          70.70        151.65   Yes  

[2 rows

In [32]:
# Agora, vou analisar separadamente as características de cada grupo para identificar diferenças significativas que possam indicar os motivos do cancelamento.
# Por exemplo, a coluna "tenure" (tempo de permanência) pode ser um fator importante. Vou calcular a média do tempo de permanência para ambos os grupos.
from scipy.stats import ttest_ind
avg_tenure_churned = churned_customers['tenure'].mean()
print("Average Tenure of Churned Customers:", avg_tenure_churned.round(2))
avg_tenure_retained = retained_customers['tenure'].mean()
print("Average Tenure of Retained Customers:", avg_tenure_retained.round(2))

# O resultado foi que os clientes que cancelaram o serviço têm, em média, um tempo de permanência significativamente menor do que os clientes que permaneceram.
# Isso sugere que o tempo de permanência pode ser um fator importante no cancelamento do serviço.

# 🤨 Mas por qual motivo os clientes novatos cancelaram o contrato? O que poderia fazer com que eles fiquem mais tempo na empresa?
# Eu vou usar o teste t-teste para verificar se a diferença entre as médias é estatisticamente significativa.


tenure_churn = churned_customers['tenure']
tenure_no_churn = retained_customers['tenure']

t_stat, p_value = ttest_ind(tenure_churn, tenure_no_churn, equal_var=False)

print("t-statistic:", t_stat)
print("p-value:", p_value)

Average Tenure of Churned Customers: 17.98
Average Tenure of Retained Customers: 37.57
t-statistic: -34.823818696312976
p-value: 1.1954945472607151e-232


In [7]:
# Qual gênero mais cancela o serviço?

sum_churned_women = churned_customers[churned_customers['gender']
                                      == 'Female'].shape[0]
print("Total de mulheres que cancelaram o serviço:", sum_churned_women)

sum_churned_man = churned_customers[churned_customers['gender']
                                    == 'Male'].shape[0]
print("Total de homens que cancelaram o serviço:", sum_churned_man)

# O resultado não parece interferir muito, mas as mulheres cancelaram um pouco mais o serviço do que os homens.
# É necessário analisar cada coluna dentro do dataset para entender quais fatores influenciam no cancelamento do serviço.
# Fonte de livro:
# “Practical Statistics for Data Scientists”, O’Reilly (Bruce et al., 2020), cap. 2–3
# → Apresenta a necessidade de análises univariadas estratificadas por classe como primeira etapa exploratória.

Total de mulheres que cancelaram o serviço: 939
Total de homens que cancelaram o serviço: 930


In [8]:
# Agora vou analisar a coluna "SeniorCitizen" para ver se há alguma diferença entre os clientes idosos e não idosos em relação ao cancelamento do serviço.

sum_churned_senior = churned_customers[churned_customers['SeniorCitizen'] == 1].shape[0]
print("Total de idosos que cancelaram o serviço:", sum_churned_senior)
sum_churned_not_senior = churned_customers[churned_customers['SeniorCitizen'] == 0].shape[0]
print("Total de não idosos que cancelaram o serviço:", sum_churned_not_senior)

# O resultado mostra que os clientes não idosos cancelaram o serviço mais vezes do que os idosos.
# Mas isso não diz nada.

# Vou descobrir quantos idosos e não idosos existem no dataset para entender melhor esses números.
total_senior = df[df['SeniorCitizen'] == 1].shape[0]
print("Total de idosos no dataset:", total_senior)
total_not_senior = df[df['SeniorCitizen'] == 0].shape[0]
print("Total de não idosos no dataset:", total_not_senior)

# Cálculo de taxas de cancelamento dos idosos e não idosos
churn_rate_senior = (sum_churned_senior / total_senior) * 100
print("Taxa de cancelamento dos idosos: {:.2f}%".format(churn_rate_senior))
churn_rate_not_senior = (sum_churned_not_senior / total_not_senior) * 100
print("Taxa de cancelamento dos não idosos: {:.2f}%".format(
    churn_rate_not_senior))

# Podemos ver que os idosos têm uma taxa de cancelamento maior do que os não idosos.
# Isso sugere que ser idoso pode ser um fator que influencia no cancelamento do serviço
# Relative Risk (RR) Risk Ratio razão de riscos

rr = churn_rate_senior / churn_rate_not_senior
print(
    "Relative Risk (RR) de idosos em relação aos não idosos: {:.2f}".format(rr))

Total de idosos que cancelaram o serviço: 476
Total de não idosos que cancelaram o serviço: 1393
Total de idosos no dataset: 1142
Total de não idosos no dataset: 5901
Taxa de cancelamento dos idosos: 41.68%
Taxa de cancelamento dos não idosos: 23.61%
Relative Risk (RR) de idosos em relação aos não idosos: 1.77


In [9]:
# Agora vou analisar a coluna 'Partner' para ver se há alguma diferença entre os clientes que têm parceiro e os que não têm em relação ao cancelamento do serviço.

import pandas as pd
import scipy
from scipy.stats import chi2_contingency

sum_churned_partner = churned_customers[churned_customers['Partner']
                                        == 'Yes'].shape[0]
print("Total de clientes com parceiro que cancelaram o serviço:", sum_churned_partner)
sum_churned_no_partner = churned_customers[churned_customers['Partner']
                                           == 'No'].shape[0]
print("Total de clientes sem parceiro que cancelaram o serviço:",
      sum_churned_no_partner)

# Vou descobrir quantos clientes com parceiro e sem parceiro existem no dataset para entender melhor esses números.
total_partner = df[df['Partner'] == 'Yes'].shape[0]
total_no_partner = df[df['Partner'] == 'No'].shape[0]
print("Total de clientes com parceiro:", total_partner)
print("Total de clientes sem parceiro:", total_no_partner)

# Cálculo de taxas de cancelamento dos clientes com parceiro e sem parceiro
churn_rate_partner = (sum_churned_partner / total_partner) * 100
churn_rate_no_partner = (sum_churned_no_partner / total_no_partner) * 100
print("Taxa de cancelamento dos clientes com parceiro:",
      churn_rate_partner)
print("Taxa de cancelamento dos clientes sem parceiro:",
      churn_rate_no_partner)

# É possível ver que os clientes sem parceiro têm uma taxa de cancelamento maior do que os clientes com parceiro.
# Isso sugere que não ter um parceiro pode ser um fator que influencia no cancelamento do serviço.
# Relative Risk (RR) Risk Ratio razão de riscos
rr_partner = churn_rate_no_partner / churn_rate_partner
print("Relative Risk (RR) de clientes sem parceiro em relação aos com parceiro: {:.2f}".format(
    rr_partner))


# Tabela de contingência
data = [
    [669, 2733],   # Partner = Yes
    [1200, 2441]   # Partner = No
]

# Rodando o teste qui-quadrado
chi2, p_value, dof, expected = chi2_contingency(data)

print("Chi-square:", chi2)
print("p-valor:", p_value)
print("Graus de liberdade:", dof)
print("Valores esperados:", expected)

# Com essas interpretações de  p-valor e chi-square, podemos concluir que há uma associação significativa entre ter um parceiro e o cancelamento do serviço.
# No caso, clientes sem parceiro têm uma taxa de cancelamento maior do que clientes com parceiro.

Total de clientes com parceiro que cancelaram o serviço: 669
Total de clientes sem parceiro que cancelaram o serviço: 1200
Total de clientes com parceiro: 3402
Total de clientes sem parceiro: 3641
Taxa de cancelamento dos clientes com parceiro: 19.66490299823633
Taxa de cancelamento dos clientes sem parceiro: 32.95797857731392
Relative Risk (RR) de clientes sem parceiro em relação aos com parceiro: 1.68
Chi-square: 158.7333820309922
p-valor: 2.1399113440759935e-36
Graus de liberdade: 1
Valores esperados: [[ 902.78830044 2499.21169956]
 [ 966.21169956 2674.78830044]]


In [10]:
# Agora, vou fazer a mesma análise para a coluna 'Dependents', que indica se o cliente tem dependentes ou não.
sum_churned_dependents = churned_customers[churned_customers['Dependents']
                                           == 'Yes'].shape[0]
print("Total de clientes com dependentes que cancelaram o serviço:",
      sum_churned_dependents)
sum_churned_no_dependents = churned_customers[churned_customers['Dependents']
                                              == 'No'].shape[0]
print("Total de clientes sem dependentes que cancelaram o serviço:",
      sum_churned_no_dependents)

# Taxa de cancelamento dos clientes com dependentes e sem dependentes
total_dependents = df[df['Dependents'] == 'Yes'].shape[0]
total_no_dependents = df[df['Dependents'] == 'No'].shape[0]
print("Total de clientes com dependentes:", total_dependents)
print("Total de clientes sem dependentes:", total_no_dependents)

# Cálculo de taxas de cancelamento dos clientes com dependentes e sem dependentes
churn_rate_dependents = (sum_churned_dependents / total_dependents) * 100
churn_rate_no_dependents = (
    sum_churned_no_dependents / total_no_dependents) * 100
print("Taxa de cancelamento dos clientes com dependentes:", churn_rate_dependents)
print("Taxa de cancelamento dos clientes sem dependentes:", churn_rate_no_dependents)

# É possível ver que os clientes sem dependentes têm uma taxa de cancelamento maior do que os clientes com dependentes.
# Isso sugere que não ter dependentes pode ser um fator que influencia no cancelamento do serviço.
# Relative Risk (RR) Risk Ratio razão de riscos
rr_dependents = churn_rate_no_dependents / churn_rate_dependents
print("Relative Risk (RR) de clientes sem dependentes em relação aos com dependentes: {:.2f}".format(
    rr_dependents))

Total de clientes com dependentes que cancelaram o serviço: 326
Total de clientes sem dependentes que cancelaram o serviço: 1543
Total de clientes com dependentes: 2110
Total de clientes sem dependentes: 4933
Taxa de cancelamento dos clientes com dependentes: 15.450236966824646
Taxa de cancelamento dos clientes sem dependentes: 31.279140482465028
Relative Risk (RR) de clientes sem dependentes em relação aos com dependentes: 2.02


In [11]:
# Agora vou analisar a coluna 'PhoneService' para ver se há alguma diferença entre os clientes que têm serviço de telefone e os que não têm em relação ao cancelamento do serviço.
sum_churned_phone_service = churned_customers[churned_customers['PhoneService']
                                              == 'Yes'].shape[0]
print("Total de clientes com serviço de telefone que cancelaram o serviço:",
      sum_churned_phone_service)
sum_churned_no_phone_service = churned_customers[churned_customers['PhoneService']
                                                 == 'No'].shape[0]
print("Total de clientes sem serviço de telefone que cancelaram o serviço:",
      sum_churned_no_phone_service)

# Vou descobrir quantos clientes com serviço de telefone e sem serviço de telefone existem no dataset para entender melhor esses números.
total_phone_service = df[df['PhoneService'] == 'Yes'].shape[0]
total_no_phone_service = df[df['PhoneService'] == 'No'].shape[0]
print("Total de clientes com serviço de telefone:", total_phone_service)
print("Total de clientes sem serviço de telefone:", total_no_phone_service)

# Cálculo de taxas de cancelamento dos clientes com serviço de telefone e sem serviço de telefone
churn_rate_phone_service = (
    sum_churned_phone_service / total_phone_service) * 100
churn_rate_no_phone_service = (
    sum_churned_no_phone_service / total_no_phone_service) * 100
print("Taxa de cancelamento dos clientes com serviço de telefone:",
      churn_rate_phone_service)
print("Taxa de cancelamento dos clientes sem serviço de telefone:",
      churn_rate_no_phone_service)

# Podemos concluir que a coluna 'PhoneService' não parece influenciar significativamente no cancelamento do serviço, já que as taxas de cancelamento são bastante semelhantes entre os dois grupos.

Total de clientes com serviço de telefone que cancelaram o serviço: 1699
Total de clientes sem serviço de telefone que cancelaram o serviço: 170
Total de clientes com serviço de telefone: 6361
Total de clientes sem serviço de telefone: 682
Taxa de cancelamento dos clientes com serviço de telefone: 26.70963684955196
Taxa de cancelamento dos clientes sem serviço de telefone: 24.926686217008797


In [12]:
# Agora vou analisar a coluna 'MultipleLines' para ver se há alguma diferença entre os clientes que têm múltiplas linhas e os que não têm em relação ao cancelamento do serviço.
sum_churned_multiple_lines = churned_customers[churned_customers['MultipleLines']
                                               == 'Yes'].shape[0]
print("Total de clientes com múltiplas linhas que cancelaram o serviço:",
      sum_churned_multiple_lines)
sum_churned_no_multiple_lines = churned_customers[churned_customers['MultipleLines']
                                                  == 'No'].shape[0]
print("Total de clientes sem múltiplas linhas que cancelaram o serviço:",
      sum_churned_no_multiple_lines)

# Vou descobrir quantos clientes com múltiplas linhas e sem múltiplas linhas existem no dataset para entender melhor esses números.
total_multiple_lines = df[df['MultipleLines'] == 'Yes'].shape[0]
total_no_multiple_lines = df[df['MultipleLines'] == 'No'].shape[0]
print("Total de clientes com múltiplas linhas:", total_multiple_lines)
print("Total de clientes sem múltiplas linhas:", total_no_multiple_lines)

# Cálculo de taxas de cancelamento dos clientes com múltiplas linhas e sem múltiplas linhas
churn_rate_multiple_lines = (
    sum_churned_multiple_lines / total_multiple_lines) * 100
churn_rate_no_multiple_lines = (
    sum_churned_no_multiple_lines / total_no_multiple_lines) * 100
print("Taxa de cancelamento dos clientes com múltiplas linhas:",
      churn_rate_multiple_lines)
print("Taxa de cancelamento dos clientes sem múltiplas linhas:",
      churn_rate_no_multiple_lines)

# Podemos concluir que a coluna 'MultipleLines' não parece influenciar significativamente no cancelamento do serviço, já que as taxas de cancelamento são bastante semelhantes entre os dois grupos.

Total de clientes com múltiplas linhas que cancelaram o serviço: 850
Total de clientes sem múltiplas linhas que cancelaram o serviço: 849
Total de clientes com múltiplas linhas: 2971
Total de clientes sem múltiplas linhas: 3390
Taxa de cancelamento dos clientes com múltiplas linhas: 28.6098956580276
Taxa de cancelamento dos clientes sem múltiplas linhas: 25.04424778761062


In [13]:
# Agora vamos analisar a coluna 'InternetService' para ver se há alguma diferença entre os clientes que têm serviço de internet e os que não têm em relação ao cancelamento do serviço.

import pandas as pd
from scipy.stats import chi2_contingency

# Tabela de contingência
contingency = pd.crosstab(df['InternetService'], df['Churn'])
print("Tabela de contingência:\n", contingency)

# Teste qui-quadrado
chi2, p, dof, expected = chi2_contingency(contingency)

print("\nChi-square:", chi2)
print("p-valor:", p)
print("Graus de liberdade:", dof)
print("Valores esperados:\n", expected)

# Taxas de churn por categoria
churn_rates = df.groupby('InternetService')['Churn'].value_counts(
    normalize=True).unstack()['Yes'] * 100
print("\nTaxa de churn por tipo de InternetService:\n", churn_rates)

# Podemos concluir que a coluna 'InternetService' influencia significativamente no cancelamento do serviço, já que as taxas de cancelamento variam bastante entre os diferentes tipos de serviço de internet.
# Clientes com serviço de internet do tipo 'Fiber optic' têm uma taxa de cancelamento muito maior do que os clientes com outros tipos de serviço de internet.

Tabela de contingência:
 Churn              No   Yes
InternetService            
DSL              1962   459
Fiber optic      1799  1297
No               1413   113

Chi-square: 732.309589667794
p-valor: 9.571788222840544e-160
Graus de liberdade: 2
Valores esperados:
 [[1778.53954281  642.46045719]
 [2274.41488002  821.58511998]
 [1121.04557717  404.95442283]]

Taxa de churn por tipo de InternetService:
 InternetService
DSL            18.959108
Fiber optic    41.892765
No              7.404980
Name: Yes, dtype: float64


In [14]:
# Agora vamos analisar a coluna 'OnlineSecurity' para ver se há alguma diferença entre os clientes que têm segurança online e os que não têm em relação ao cancelamento do serviço.

sum_churned_online_security = churned_customers[churned_customers['OnlineSecurity']
                                                == 'Yes'].shape[0]
print("Total de clientes com segurança online que cancelaram o serviço:",
      sum_churned_online_security)
sum_churned_no_online_security = churned_customers[churned_customers['OnlineSecurity']
                                                   == 'No'].shape[0]
print("Total de clientes sem segurança online que cancelaram o serviço:",
      sum_churned_no_online_security)

# Vou descobrir quantos clientes com segurança online e sem segurança online existem no dataset para entender melhor esses números.
total_online_security = df[df['OnlineSecurity'] == 'Yes'].shape[0]
total_no_online_security = df[df['OnlineSecurity'] == 'No'].shape[0]
print("Total de clientes com segurança online:", total_online_security)
print("Total de clientes sem segurança online:", total_no_online_security)

# Taxa de cancelamento dos clientes com segurança online e sem segurança online
churn_rate_online_security = (
    sum_churned_online_security / total_online_security) * 100
churn_rate_no_online_security = (
    sum_churned_no_online_security / total_no_online_security) * 100
print("Taxa de cancelamento dos clientes com segurança online:",
      churn_rate_online_security)
print("Taxa de cancelamento dos clientes sem segurança online:",
      churn_rate_no_online_security)

# Podemos concluir que a coluna 'OnlineSecurity' influencia significativamente no cancelamento do serviço, já que os clientes sem segurança online têm uma taxa de cancelamento muito maior do que os clientes com segurança online.

Total de clientes com segurança online que cancelaram o serviço: 295
Total de clientes sem segurança online que cancelaram o serviço: 1461
Total de clientes com segurança online: 2019
Total de clientes sem segurança online: 3498
Taxa de cancelamento dos clientes com segurança online: 14.611193660227837
Taxa de cancelamento dos clientes sem segurança online: 41.76672384219554


In [15]:
# Agora vou analisar a coluna 'OnlineBackup' para ver se há alguma diferença entre os clientes que têm backup online e os que não têm em relação ao cancelamento do serviço.
sum_churned_online_backup = churned_customers[churned_customers['OnlineBackup']
                                              == 'Yes'].shape[0]
print("Total de clientes com backup online que cancelaram o serviço:",
      sum_churned_online_backup)
sum_churned_no_online_backup = churned_customers[churned_customers['OnlineBackup']
                                                 == 'No'].shape[0]
print("Total de clientes sem backup online que cancelaram o serviço:",
      sum_churned_no_online_backup)

# Vou descobrir quantos clientes com backup online e sem backup online existem no dataset para entender melhor esses números.
total_online_backup = df[df['OnlineBackup'] == 'Yes'].shape[0]
total_no_online_backup = df[df['OnlineBackup'] == 'No'].shape[0]
print("Total de clientes com backup online:", total_online_backup)
print("Total de clientes sem backup online:", total_no_online_backup)

# Cálculo de taxas de cancelamento dos clientes com backup online e sem backup online
churn_rate_online_backup = (
    sum_churned_online_backup / total_online_backup) * 100
churn_rate_no_online_backup = (
    sum_churned_no_online_backup / total_no_online_backup) * 100
print("Taxa de cancelamento dos clientes com backup online:",
      churn_rate_online_backup)
print("Taxa de cancelamento dos clientes sem backup online:",
      churn_rate_no_online_backup)

# Podemos concluir que a coluna 'OnlineBackup' influencia significativamente no cancelamento do serviço, já que os clientes sem backup online têm uma taxa de cancelamento maior do que os clientes com backup online.

Total de clientes com backup online que cancelaram o serviço: 523
Total de clientes sem backup online que cancelaram o serviço: 1233
Total de clientes com backup online: 2429
Total de clientes sem backup online: 3088
Taxa de cancelamento dos clientes com backup online: 21.531494442157268
Taxa de cancelamento dos clientes sem backup online: 39.92875647668394


In [16]:
# Agora vou analisar a coluna de 'DeviceProtection' para ver se há alguma diferença entre os clientes que têm proteção de dispositivo e os que não têm em relação ao cancelamento do serviço.
sum_churned_device_protection = churned_customers[churned_customers['DeviceProtection']
                                                  == 'Yes'].shape[0]
print("Total de clientes com proteção de dispositivo que cancelaram o serviço:",
      sum_churned_device_protection)
sum_churned_no_device_protection = churned_customers[churned_customers['DeviceProtection']
                                                     == 'No'].shape[0]
print("Total de clientes sem proteção de dispositivo que cancelaram o serviço:",
      sum_churned_no_device_protection)

# Vou descobrir quantos clientes com proteção de dispositivo e sem proteção de dispositivo existem no dataset para entender melhor esses números.
total_device_protection = df[df['DeviceProtection'] == 'Yes'].shape[0]
total_no_device_protection = df[df['DeviceProtection'] == 'No'].shape[0]
print("Total de clientes com proteção de dispositivo:", total_device_protection)
print("Total de clientes sem proteção de dispositivo:", total_no_device_protection)

# Cálculo de taxas de cancelamento dos clientes com proteção de dispositivo e sem proteção de dispositivo
churn_rate_device_protection = (
    sum_churned_device_protection / total_device_protection) * 100
churn_rate_no_device_protection = (
    sum_churned_no_device_protection / total_no_device_protection) * 100
print("Taxa de cancelamento dos clientes com proteção de dispositivo:",
      churn_rate_device_protection)
print("Taxa de cancelamento dos clientes sem proteção de dispositivo:",
      churn_rate_no_device_protection)

# Podemos concluir que a coluna 'DeviceProtection' influencia significativamente no cancelamento do serviço, já que os clientes sem proteção de dispositivo têm uma taxa de cancelamento maior do que os clientes com proteção de dispositivo.

Total de clientes com proteção de dispositivo que cancelaram o serviço: 545
Total de clientes sem proteção de dispositivo que cancelaram o serviço: 1211
Total de clientes com proteção de dispositivo: 2422
Total de clientes sem proteção de dispositivo: 3095
Taxa de cancelamento dos clientes com proteção de dispositivo: 22.50206440957886
Taxa de cancelamento dos clientes sem proteção de dispositivo: 39.127625201938606


In [17]:
# Agora, vou analisar a coluna 'TechSupport' para ver se há alguma diferença entre os clientes que têm suporte técnico e os que não têm em relação ao cancelamento do serviço.
sum_churned_tech_support = churned_customers[churned_customers['TechSupport']
                                             == 'Yes'].shape[0]
print("Total de clientes com suporte técnico que cancelaram o serviço:",
      sum_churned_tech_support)
sum_churned_no_tech_support = churned_customers[churned_customers['TechSupport']
                                                == 'No'].shape[0]
print("Total de clientes sem suporte técnico que cancelaram o serviço:",
      sum_churned_no_tech_support)

# Vou descobrir quantos clientes com suporte técnico e sem suporte técnico existem no dataset para entender melhor esses números.
total_tech_support = df[df['TechSupport'] == 'Yes'].shape[0]
total_no_tech_support = df[df['TechSupport'] == 'No'].shape[0]
print("Total de clientes com suporte técnico:", total_tech_support)
print("Total de clientes sem suporte técnico:", total_no_tech_support)

# Cálculo de taxas de cancelamento dos clientes com suporte técnico e sem suporte técnico
churn_rate_tech_support = (
    sum_churned_tech_support / total_tech_support) * 100
churn_rate_no_tech_support = (
    sum_churned_no_tech_support / total_no_tech_support) * 100
print("Taxa de cancelamento dos clientes com suporte técnico:",
      churn_rate_tech_support)
print("Taxa de cancelamento dos clientes sem suporte técnico:",
      churn_rate_no_tech_support)

# Podemos concluir que a coluna 'TechSupport' influencia significativamente no cancelamento do serviço, já que os clientes sem suporte técnico têm uma taxa de cancelamento maior do que os clientes com suporte técnico.

Total de clientes com suporte técnico que cancelaram o serviço: 310
Total de clientes sem suporte técnico que cancelaram o serviço: 1446
Total de clientes com suporte técnico: 2044
Total de clientes sem suporte técnico: 3473
Taxa de cancelamento dos clientes com suporte técnico: 15.166340508806261
Taxa de cancelamento dos clientes sem suporte técnico: 41.63547365390153


In [18]:
# Agora vou analisar a coluna 'StreamingTV' para ver se há alguma diferença entre os clientes que têm streaming de TV e os que não têm em relação ao cancelamento do serviço.
import pandas as pd
import scipy.stats as stats
sum_churned_streaming_tv = churned_customers[churned_customers['StreamingTV']
                                             == 'Yes'].shape[0]
print("Total de clientes com streaming de TV que cancelaram o serviço:",
      sum_churned_streaming_tv)
sum_churned_no_streaming_tv = churned_customers[churned_customers['StreamingTV']
                                                == 'No'].shape[0]
print("Total de clientes sem streaming de TV que cancelaram o serviço:",
      sum_churned_no_streaming_tv)

# Vou descobrir quantos clientes com streaming de TV e sem streaming de TV existem no dataset para entender melhor esses números.
total_streaming_tv = df[df['StreamingTV'] == 'Yes'].shape[0]
total_no_streaming_tv = df[df['StreamingTV'] == 'No'].shape[0
                                                            ]
print("Total de clientes com streaming de TV:", total_streaming_tv)
print("Total de clientes sem streaming de TV:", total_no_streaming_tv)

# Cálculo de taxas de cancelamento dos clientes com streaming de TV e sem streaming de TV
churn_rate_streaming_tv = (
    sum_churned_streaming_tv / total_streaming_tv) * 100
churn_rate_no_streaming_tv = (
    sum_churned_no_streaming_tv / total_no_streaming_tv) * 100
print("Taxa de cancelamento dos clientes com streaming de TV:",
      churn_rate_streaming_tv)
print("Taxa de cancelamento dos clientes sem streaming de TV:",
      churn_rate_no_streaming_tv)

# Podemos concluir que a coluna 'StreamingTV' influencia significativamente no cancelamento do serviço, já que os clientes sem streaming de TV têm uma taxa de cancelamento maior do que os clientes com streaming de TV.


# Tabela de contingência
contingency_table = pd.crosstab(df['StreamingTV'], df['Churn'])

print("Tabela de contingência:\n", contingency_table)

# Teste qui-quadrado
chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

print("\nChi-square:", chi2)
print("p-valor:", p_value)
print("Graus de liberdade:", dof)
print("Valores esperados:\n", expected)

# Interpretação simples
if p_value < 0.0001:
    print("\n📌 Conclusão: p < 0.0001 → A associação entre StreamingTV e Churn é estatisticamente MUITO significativa.")
else:
    print("\n📌 Conclusão: p >= 0.0001 → Não há evidência estatística forte de associação.")

Total de clientes com streaming de TV que cancelaram o serviço: 814
Total de clientes sem streaming de TV que cancelaram o serviço: 942
Total de clientes com streaming de TV: 2707
Total de clientes sem streaming de TV: 2810
Taxa de cancelamento dos clientes com streaming de TV: 30.070188400443293
Taxa de cancelamento dos clientes sem streaming de TV: 33.52313167259787
Tabela de contingência:
 Churn                  No  Yes
StreamingTV                   
No                   1868  942
No internet service  1413  113
Yes                  1893  814

Chi-square: 374.2039433109813
p-valor: 5.528994485739183e-82
Graus de liberdade: 2
Valores esperados:
 [[2064.31066307  745.68933693]
 [1121.04557717  404.95442283]
 [1988.64375976  718.35624024]]

📌 Conclusão: p < 0.0001 → A associação entre StreamingTV e Churn é estatisticamente MUITO significativa.


In [19]:
# Agora vou analisar a coluna 'StreamingMovies' para ver se há alguma diferença entre os clientes que têm streaming de filmes e os que não têm em relação ao cancelamento do serviço.
import pandas as pd
import scipy.stats as stats
sum_churned_streaming_movies = churned_customers[churned_customers['StreamingMovies']
                                                 == 'Yes'].shape[0]
print("Total de clientes com streaming de filmes que cancelaram o serviço:",
      sum_churned_streaming_movies)
sum_churned_no_streaming_movies = churned_customers[churned_customers['StreamingMovies']
                                                    == 'No'].shape[0]
print("Total de clientes sem streaming de filmes que cancelaram o serviço:",
      sum_churned_no_streaming_movies)
# Vou descobrir quantos clientes com streaming de filmes e sem streaming de filmes existem no dataset para entender melhor esses números.
total_streaming_movies = df[df['StreamingMovies'] == 'Yes'].shape[0]
total_no_streaming_movies = df[df['StreamingMovies'] == 'No'].shape[0]
print("Total de clientes com streaming de filmes:", total_streaming_movies)
print("Total de clientes sem streaming de filmes:", total_no_streaming_movies)
# Cálculo de taxas de cancelamento dos clientes com streaming de filmes e sem streaming de filmes
churn_rate_streaming_movies = (
    sum_churned_streaming_movies / total_streaming_movies) * 100
churn_rate_no_streaming_movies = (
    sum_churned_no_streaming_movies / total_no_streaming_movies) * 100
print("Taxa de cancelamento dos clientes com streaming de filmes:",
      churn_rate_streaming_movies)
print("Taxa de cancelamento dos clientes sem streaming de filmes:",
      churn_rate_no_streaming_movies)

# Podemos concluir que a coluna 'StreamingMovies' influencia significativamente no cancelamento do serviço, já que os clientes sem streaming de filmes têm uma taxa de cancelamento maior do que os clientes com streaming de filmes.

Total de clientes com streaming de filmes que cancelaram o serviço: 818
Total de clientes sem streaming de filmes que cancelaram o serviço: 938
Total de clientes com streaming de filmes: 2732
Total de clientes sem streaming de filmes: 2785
Taxa de cancelamento dos clientes com streaming de filmes: 29.941434846266475
Taxa de cancelamento dos clientes sem streaming de filmes: 33.680430879712745


In [20]:
# Agora vou analisar a coluna 'Contract' para ver se há alguma diferença entre os clientes que têm contrato mensal, anual ou bienal em relação ao cancelamento do serviço.

import pandas as pd
import scipy.stats as stats
sum_churned_month_to_month = churned_customers[churned_customers['Contract']
                                               == 'Month-to-month'].shape[0]
print("Total de clientes com contrato mensal que cancelaram o serviço:",
      sum_churned_month_to_month)
sum_churned_one_year = churned_customers[churned_customers['Contract']
                                         == 'One year'].shape[0]
print("Total de clientes com contrato anual que cancelaram o serviço:",
      sum_churned_one_year)
sum_churned_two_year = churned_customers[churned_customers['Contract']
                                         == 'Two year'].shape[0]
print("Total de clientes com contrato bienal que cancelaram o serviço:",
      sum_churned_two_year)
# Vou descobrir quantos clientes com cada tipo de contrato existem no dataset para entender melhor esses números.
total_month_to_month = df[df['Contract'] == 'Month-to-month'].shape[0]
total_one_year = df[df['Contract'] == 'One year'].shape[0]
total_two_year = df[df['Contract'] == 'Two year'].shape[0]
print("Total de clientes com contrato mensal:", total_month_to_month)
print("Total de clientes com contrato anual:", total_one_year)
print("Total de clientes com contrato bienal:", total_two_year)

# Cálculo de taxas de cancelamento dos clientes com cada tipo de contrato
churn_rate_month_to_month = (
    sum_churned_month_to_month / total_month_to_month) * 100
churn_rate_one_year = (sum_churned_one_year / total_one_year) * 100
churn_rate_two_year = (sum_churned_two_year / total_two_year) * 100
print("Taxa de cancelamento dos clientes com contrato mensal:",
      churn_rate_month_to_month)
print("Taxa de cancelamento dos clientes com contrato anual:", churn_rate_one_year)
print("Taxa de cancelamento dos clientes com contrato bienal:", churn_rate_two_year)

# Podemos concluir que a coluna 'Contract' influencia significativamente no cancelamento do serviço, já que os clientes com contrato mensal têm uma taxa de cancelamento muito maior do que os clientes com contratos anuais ou bienais.

Total de clientes com contrato mensal que cancelaram o serviço: 1655
Total de clientes com contrato anual que cancelaram o serviço: 166
Total de clientes com contrato bienal que cancelaram o serviço: 48
Total de clientes com contrato mensal: 3875
Total de clientes com contrato anual: 1473
Total de clientes com contrato bienal: 1695
Taxa de cancelamento dos clientes com contrato mensal: 42.70967741935484
Taxa de cancelamento dos clientes com contrato anual: 11.269517990495586
Taxa de cancelamento dos clientes com contrato bienal: 2.831858407079646


In [21]:
# Agora vou analisar a coluna 'PaperlessBilling' para ver se há alguma diferença entre os clientes que têm faturamento sem papel e os que não têm em relação ao cancelamento do serviço.
import pandas as pd
import scipy.stats as stats
sum_churned_paperless_billing = churned_customers[churned_customers['PaperlessBilling']
                                                  == 'Yes'].shape[0]
print("Total de clientes com faturamento sem papel que cancelaram o serviço:",
      sum_churned_paperless_billing)
sum_churned_no_paperless_billing = churned_customers[churned_customers['PaperlessBilling']
                                                     == 'No'].shape[0]
print("Total de clientes com faturamento com papel que cancelaram o serviço:",
      sum_churned_no_paperless_billing)
# Vou descobrir quantos clientes com faturamento sem papel e com papel existem no dataset para entender melhor esses números.
total_paperless_billing = df[df['PaperlessBilling'] == 'Yes'].shape[0]
total_no_paperless_billing = df[df['PaperlessBilling'] == 'No'].shape[0]
print("Total de clientes com faturamento sem papel:", total_paperless_billing)
print("Total de clientes com faturamento com papel:", total_no_paperless_billing)

# Cálculo de taxas de cancelamento dos clientes com faturamento sem papel e com papel
churn_rate_paperless_billing = (
    sum_churned_paperless_billing / total_paperless_billing) * 100
churn_rate_no_paperless_billing = (
    sum_churned_no_paperless_billing / total_no_paperless_billing) * 100
print("Taxa de cancelamento dos clientes com faturamento sem papel:",
      churn_rate_paperless_billing)
print("Taxa de cancelamento dos clientes com faturamento com papel:",
      churn_rate_no_paperless_billing)

# Podemos concluir que a coluna 'PaperlessBilling' influencia significativamente no cancelamento do serviço, já que os clientes com faturamento sem papel têm uma taxa de cancelamento maior do que os clientes com faturamento com papel.

Total de clientes com faturamento sem papel que cancelaram o serviço: 1400
Total de clientes com faturamento com papel que cancelaram o serviço: 469
Total de clientes com faturamento sem papel: 4171
Total de clientes com faturamento com papel: 2872
Taxa de cancelamento dos clientes com faturamento sem papel: 33.565092304003834
Taxa de cancelamento dos clientes com faturamento com papel: 16.33008356545961


In [22]:
# Agora vou analisar a coluna 'PaymentMethod' para ver se há alguma diferença entre os clientes que utilizam diferentes métodos de pagamento em relação ao cancelamento do serviço.
import pandas as pd
import scipy.stats as stats
sum_churned_electronic_check = churned_customers[churned_customers['PaymentMethod']
                                                 == 'Electronic check'].shape[0]
print("Total de clientes que pagam por cheque eletrônico que cancelaram o serviço:",
      sum_churned_electronic_check)
sum_churned_mailed_check = churned_customers[churned_customers['PaymentMethod']
                                             == 'Mailed check'].shape[0]
print("Total de clientes que pagam por cheque enviado pelo correio que cancelaram o serviço:",
      sum_churned_mailed_check)
sum_churned_bank_transfer = churned_customers[churned_customers['PaymentMethod']
                                              == 'Bank transfer (automatic)'].shape[0]
print("Total de clientes que pagam por transferência bancária automática que cancelaram o serviço:",
      sum_churned_bank_transfer)
sum_churned_credit_card = churned_customers[churned_customers['PaymentMethod']
                                            == 'Credit card (automatic)'].shape[0]
print("Total de clientes que pagam por cartão de crédito automático que cancelaram o serviço:",
      sum_churned_credit_card)
# Vou descobrir quantos clientes com cada método de pagamento existem no dataset para entender melhor esses números.
total_electronic_check = df[df['PaymentMethod'] == 'Electronic check'].shape[0]
total_mailed_check = df[df['PaymentMethod'] == 'Mailed check'].shape[0]
total_bank_transfer = df[df['PaymentMethod']
                         == 'Bank transfer (automatic)'].shape[0]
total_credit_card = df[df['PaymentMethod']
                       == 'Credit card (automatic)'].shape[0]
print("Total de clientes que pagam por cheque eletrônico:", total_electronic_check)
print("Total de clientes que pagam por cheque enviado pelo correio:", total_mailed_check)
print("Total de clientes que pagam por transferência bancária automática:",
      total_bank_transfer)
print("Total de clientes que pagam por cartão de crédito automático:", total_credit_card)
# Cálculo de taxas de cancelamento dos clientes com cada método de pagamento
churn_rate_electronic_check = (
    sum_churned_electronic_check / total_electronic_check) * 100
churn_rate_mailed_check = (
    sum_churned_mailed_check / total_mailed_check) * 100
churn_rate_bank_transfer = (
    sum_churned_bank_transfer / total_bank_transfer) * 100
churn_rate_credit_card = (
    sum_churned_credit_card / total_credit_card) * 100
print("Taxa de cancelamento dos clientes que pagam por cheque eletrônico:",
      churn_rate_electronic_check)
print("Taxa de cancelamento dos clientes que pagam por cheque enviado pelo correio:",
      churn_rate_mailed_check)
print("Taxa de cancelamento dos clientes que pagam por transferência bancária automática:",
      churn_rate_bank_transfer)
print("Taxa de cancelamento dos clientes que pagam por cartão de crédito automático:",
      churn_rate_credit_card)
# Podemos concluir que a coluna 'PaymentMethod' influencia significativamente no cancelamento do serviço, já que os clientes que pagam por cheque eletrônico têm uma taxa de cancelamento muito maior do que os clientes que utilizam outros métodos de pagamento.

Total de clientes que pagam por cheque eletrônico que cancelaram o serviço: 1071
Total de clientes que pagam por cheque enviado pelo correio que cancelaram o serviço: 308
Total de clientes que pagam por transferência bancária automática que cancelaram o serviço: 258
Total de clientes que pagam por cartão de crédito automático que cancelaram o serviço: 232
Total de clientes que pagam por cheque eletrônico: 2365
Total de clientes que pagam por cheque enviado pelo correio: 1612
Total de clientes que pagam por transferência bancária automática: 1544
Total de clientes que pagam por cartão de crédito automático: 1522
Taxa de cancelamento dos clientes que pagam por cheque eletrônico: 45.28541226215645
Taxa de cancelamento dos clientes que pagam por cheque enviado pelo correio: 19.106699751861044
Taxa de cancelamento dos clientes que pagam por transferência bancária automática: 16.709844559585495
Taxa de cancelamento dos clientes que pagam por cartão de crédito automático: 15.243101182654403


In [23]:
# Agora vou analisar a coluna 'MonthlyCharges' para ver se há alguma diferença nas cobranças mensais entre os clientes que cancelaram o serviço e os que não cancelaram.
import pandas as pd

churned_customers = df[df['Churn'] == 'Yes']
not_churned_customers = df[df['Churn'] == 'No']

avg_monthly_charges_churned = churned_customers['MonthlyCharges'].mean()
print("Média das cobranças mensais dos clientes que cancelaram o serviço: R${:.2f}".format(
    avg_monthly_charges_churned))
avg_monthly_charges_not_churned = not_churned_customers['MonthlyCharges'].mean(
)
print("Média das cobranças mensais dos clientes que não cancelaram o serviço: R${:.2f}".format(
    avg_monthly_charges_not_churned))


# Podemos ver que os clientes que cancelaram o serviço têm uma média de cobranças mensais maior do que os clientes que não cancelaram.
# Isso sugere que cobranças mensais mais altas podem ser um fator que influencia no cancelamento do serviço.

Média das cobranças mensais dos clientes que cancelaram o serviço: R$74.44
Média das cobranças mensais dos clientes que não cancelaram o serviço: R$61.27


In [24]:
# Agora vou analisar a coluna 'TotalCharges'  para ver se há alguma diferença nas cobranças totais entre os clientes que cancelaram o serviço e os que não cancelaram.
import pandas as pd
churned_customers = df[df['Churn'] == 'Yes']
not_churned_customers = df[df['Churn'] == 'No']

avg_total_charges_churned_total = churned_customers['TotalCharges'].mean()
print("Média das cobranças totais dos clientes que cancelaram o serviço: R${:.2f}".format(
    avg_total_charges_churned_total))


avg_total_charges_not_churned_total = not_churned_customers['TotalCharges'].mean(
)
print("Média das cobranças totais dos clientes que não cancelaram o serviço: R${:.2f}".format(
    avg_total_charges_not_churned_total))

TypeError: Could not convert string '108.15151.65820.53046.055036.3528.3539.6520.154749.1530.21093.1316.93549.251105.4144.151426.4633.31752.55857.2579.35496.9927.35113.852514.57611.855238.92868.151064.65229.552135.51502.6535.4581.2570.45563.6545.65655.51592.352732651.2242.85000.21145.7503.629.95662.651111.6524.853179.91934.4584.5268.35179.3555.284.6114.11120.31074.348.55147.1540.280.55132.21364.31099.61511.224.870.6401.15624.851779.9570606.55435.41712.773.6396.12303.35415.553058.652231.054350.1139.05511.25155.85293.954759.856603168.2371.9840.1846805.1492.55181.52566.550.6590.85214.754874.72524.457007.619.919.6233.71845.975.31182.55518.96396.4569.552227.1385.9673.25734.355574.71861.5453.4994.82145680.0574.35139.4903.6131.051238.451782.4167.5502.85334.84131.951929.351758.6348085834.159732662596.153527.65614.45233.5575.3346.285.7713.756786.4248.95663.05137.64547.254687.96985.655794.65855.32499.389.551899.6545.7161.5222.374.6164.335.996.05145.15191.0575.052550.962.258091.1270.7168.152015.3

In [ ]:
# Agora eu vou criar um modelo
# Apenas lembrando que a análise de cada coluna foi feita de forma isolada, sem considerar possíveis interações entre as variáveis.
# Porém, na prática, essas variáveis podem interagir entre si e influenciar o comportamento de cancelamento dos clientes de maneiras mais complexas.
# Portanto, farei uma análise estatística mais avançada utilizando um modelo de regressão logística para entender melhor esses fatores e suas interações.